In [3]:
import os
import pandas as pd

def load_existing_songs(library_file):
    """Carga las canciones existentes en el archivo Excel en un conjunto para compararlas."""
    library_df_dict = pd.read_excel(library_file, sheet_name=None)
    existing_songs = set()

    for sheet_name, library_df in library_df_dict.items():
        if 'Ubicacion' in library_df.columns and 'Nombre' in library_df.columns:
            for _, row in library_df.iterrows():
                file_path = os.path.join(row['Ubicacion'], row['Nombre'] + ".mp3")
                existing_songs.add(file_path)

    return existing_songs, library_df_dict

def find_new_songs(directory, existing_songs):
    """Busca canciones en el directorio y subdirectorios que no están en el conjunto existing_songs."""
    new_songs = []

    for root, _, files in os.walk(directory):
        for file_name in files:
            if file_name.lower().endswith('.mp3'):
                file_path = os.path.join(root, file_name)
                if file_path not in existing_songs:
                    new_songs.append((root, file_name.replace(".mp3", "")))

    return new_songs

def add_new_songs_to_excel(library_file, new_songs, library_df_dict):
    """Agrega las nuevas canciones al archivo Excel con las columnas dadas y guarda los cambios."""
    added_songs = []  # Para almacenar las canciones nuevas que se agregarán
    
    for root, file_name in new_songs:
        # Agregar la nueva canción a la primera hoja por simplicidad
        sheet_name = list(library_df_dict.keys())[0]
        
        # Crear una nueva fila con las columnas especificadas
        new_row = {
            'Nombre': file_name,
            'Titulo': file_name,
            'Artista': "",  # Dejar en blanco
            'Genero': "",  # Dejar en blanco
            'Ubicacion': root,
            'BPM': "",  # Dejar en blanco
            'Clave / Key': "",  # Dejar en blanco
            'Pais': "",  # Dejar en blanco
            'Valoracion': "",  # Dejar en blanco
            'Comentario': ""  # Dejar en blanco
        }
        
        # Agregar la nueva fila al DataFrame correspondiente
        library_df_dict[sheet_name] = pd.concat([library_df_dict[sheet_name], pd.DataFrame([new_row])], ignore_index=True)
        
        # Añadir a la lista de canciones añadidas
        added_songs.append(new_row)

    # Guardar los cambios en el archivo Excel
    with pd.ExcelWriter(library_file, engine='openpyxl') as writer:
        for sheet_name, df in library_df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Archivo Excel actualizado con nuevas canciones: {library_file}")

    return added_songs

# Configuración
library_file = r"E:\Musica-E\Organizacion\Biblioteca.xlsx"
search_directory = r"E:\Musica-E\DJ Project"

# Cargar las canciones existentes en el archivo Excel
existing_songs, library_df_dict = load_existing_songs(library_file)

# Buscar nuevas canciones en el directorio
new_songs = find_new_songs(search_directory, existing_songs)

# Agregar las nuevas canciones al archivo Excel
if new_songs:
    added_songs = add_new_songs_to_excel(library_file, new_songs, library_df_dict)
    print("\nNuevas canciones añadidas al archivo Excel:")
    for song in added_songs:
        print(f"Nombre: {song['Nombre']}, Título: {song['Titulo']}, Ubicación: {song['Ubicacion']}")
else:
    print("No se encontraron canciones nuevas para agregar.")


Archivo Excel actualizado con nuevas canciones: E:\Musica-E\Organizacion\Biblioteca.xlsx

Nuevas canciones añadidas al archivo Excel:
Nombre: Nutbush City Limit (90s versio... - tina turner, Título: Nutbush City Limit (90s versio... - tina turner, Ubicación: E:\Musica-E\DJ Project\90s
Nombre: David Guetta, Anne-Marie, Coi Leray - Baby Don’t Hurt Me (Lyric video), Título: David Guetta, Anne-Marie, Coi Leray - Baby Don’t Hurt Me (Lyric video), Ubicación: E:\Musica-E\DJ Project\Biblioteca\Nuevos
Nombre: MC Kevinho - Olha A Explosao, Título: MC Kevinho - Olha A Explosao, Ubicación: E:\Musica-E\DJ Project\Brasil
Nombre: Bruno Mars - That’s What I Like, Título: Bruno Mars - That’s What I Like, Ubicación: E:\Musica-E\DJ Project\dance music
Nombre: Don’t Stop The Party - Pitbull Ft TJR, Título: Don’t Stop The Party - Pitbull Ft TJR, Ubicación: E:\Musica-E\DJ Project\dance music
Nombre: Vente Pa’ Ca - Ricky Martin Ft Maluma, Título: Vente Pa’ Ca - Ricky Martin Ft Maluma, Ubicación: E:\Musica-E\